# Experiment 004: Lattice-Based Packing for Large N

Implement lattice-based packing approach for large N values (72, 100, 110, 144, 156, 196, 200).
The idea is to start with 2 base trees in an optimal configuration, then translate them in a grid pattern.

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree
import random
import math
from numba import njit

getcontext().prec = 25

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])
TREE_VERTICES = list(zip(TX, TY))

print("Tree geometry loaded")

Tree geometry loaded


In [2]:
class ChristmasTree:
    def __init__(self, center_x=0.0, center_y=0.0, angle=0.0):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        self._update_polygon()
    
    def _update_polygon(self):
        initial_polygon = Polygon(TREE_VERTICES)
        rotated = affinity.rotate(initial_polygon, self.angle, origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)
    
    def set_params(self, x, y, angle):
        self.center_x = float(x)
        self.center_y = float(y)
        self.angle = float(angle)
        self._update_polygon()
    
    def clone(self):
        return ChristmasTree(self.center_x, self.center_y, self.angle)

def has_collision(trees):
    """Check for collisions between trees"""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        candidates = tree_index.query(poly)
        for j in candidates:
            if i < j and poly.intersects(polygons[j]) and not poly.touches(polygons[j]):
                return True
    return False

def compute_bounding_side(trees):
    """Compute bounding box side length"""
    all_points = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_points.append(coords)
    all_points = np.vstack(all_points)
    side = max(all_points.max(axis=0) - all_points.min(axis=0))
    return side

def compute_score(trees):
    """Compute score contribution for N trees"""
    n = len(trees)
    side = compute_bounding_side(trees)
    return side**2 / n

print("Classes defined")

Classes defined


In [3]:
def create_lattice_config(base_trees, nx, ny, dx, dy, target_n):
    """
    Create a lattice configuration by translating base trees.
    
    Args:
        base_trees: List of base tree configurations (usually 2 trees)
        nx, ny: Grid dimensions
        dx, dy: Translation distances in x and y
        target_n: Target number of trees
    
    Returns:
        List of trees in lattice configuration
    """
    trees = []
    for tree in base_trees:
        for ix in range(nx):
            for iy in range(ny):
                new_tree = ChristmasTree(
                    tree.center_x + ix * dx,
                    tree.center_y + iy * dy,
                    tree.angle
                )
                trees.append(new_tree)
                if len(trees) >= target_n:
                    return trees[:target_n]
    return trees[:target_n]

def optimize_base_config(n_base, target_n, nx, ny, iterations=1000, seed=42):
    """
    Optimize base tree configuration using simulated annealing.
    
    Args:
        n_base: Number of base trees (usually 2)
        target_n: Target number of trees in final config
        nx, ny: Grid dimensions
        iterations: Number of SA iterations
        seed: Random seed
    """
    random.seed(seed)
    np.random.seed(seed)
    
    # Initialize base trees with random positions and angles
    base_trees = []
    for i in range(n_base):
        x = random.uniform(-0.5, 0.5)
        y = random.uniform(-0.5, 0.5)
        angle = random.uniform(0, 360)
        base_trees.append(ChristmasTree(x, y, angle))
    
    # Initial translation distances (will be optimized)
    dx = 0.6  # Approximate tree width
    dy = 0.6
    
    # Create initial config
    trees = create_lattice_config(base_trees, nx, ny, dx, dy, target_n)
    
    if has_collision(trees):
        # Increase spacing until no collision
        while has_collision(trees) and dx < 2.0:
            dx += 0.05
            dy += 0.05
            trees = create_lattice_config(base_trees, nx, ny, dx, dy, target_n)
    
    best_score = compute_score(trees)
    best_base = [t.clone() for t in base_trees]
    best_dx, best_dy = dx, dy
    
    T = 0.1  # Initial temperature
    T_min = 0.0001
    alpha = 0.995
    
    for it in range(iterations):
        # Perturb base trees
        new_base = [t.clone() for t in base_trees]
        idx = random.randint(0, n_base - 1)
        
        move_type = random.choice(['position', 'angle', 'spacing'])
        
        if move_type == 'position':
            new_base[idx].set_params(
                new_base[idx].center_x + random.uniform(-0.05, 0.05),
                new_base[idx].center_y + random.uniform(-0.05, 0.05),
                new_base[idx].angle
            )
        elif move_type == 'angle':
            new_base[idx].set_params(
                new_base[idx].center_x,
                new_base[idx].center_y,
                (new_base[idx].angle + random.uniform(-10, 10)) % 360
            )
        else:  # spacing
            new_dx = dx + random.uniform(-0.02, 0.02)
            new_dy = dy + random.uniform(-0.02, 0.02)
            if new_dx > 0.3 and new_dy > 0.3:
                dx, dy = new_dx, new_dy
        
        # Create new config
        trees = create_lattice_config(new_base, nx, ny, dx, dy, target_n)
        
        if has_collision(trees):
            continue
        
        new_score = compute_score(trees)
        delta = new_score - best_score
        
        if delta < 0 or random.random() < math.exp(-delta / T):
            base_trees = new_base
            if new_score < best_score:
                best_score = new_score
                best_base = [t.clone() for t in base_trees]
                best_dx, best_dy = dx, dy
                if it % 100 == 0:
                    print(f"  Iter {it}: New best score = {best_score:.6f}")
        
        T = max(T * alpha, T_min)
    
    # Return best configuration
    return create_lattice_config(best_base, nx, ny, best_dx, best_dy, target_n), best_score

print("Lattice functions defined")

Lattice functions defined


In [4]:
# Load baseline submission to compare
baseline_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv')

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = float(parse_value(row['x']))
        y = float(parse_value(row['y']))
        deg = float(parse_value(row['deg']))
        trees.append(ChristmasTree(x, y, deg))
    return trees

# Get baseline scores for target N values
target_ns = [72, 100, 110, 144, 156, 196, 200]
baseline_scores = {}
for n in target_ns:
    trees = load_trees_for_n(baseline_df, n)
    baseline_scores[n] = compute_score(trees)
    print(f"N={n}: baseline score = {baseline_scores[n]:.6f}")

N=72: baseline score = 0.348559
N=100: baseline score = 0.345531
N=110: baseline score = 0.337604
N=144: baseline score = 0.342276
N=156: baseline score = 0.329987
N=196: baseline score = 0.333299
N=200: baseline score = 0.337731


In [5]:
# Grid configurations for each target N
# For N trees with 2 base trees, we need nx * ny >= N/2
grid_configs = {
    72: (6, 6),    # 6*6*2 = 72
    100: (5, 10),  # 5*10*2 = 100
    110: (5, 11),  # 5*11*2 = 110 (take first 110)
    144: (6, 12),  # 6*12*2 = 144
    156: (6, 13),  # 6*13*2 = 156
    196: (7, 14),  # 7*14*2 = 196
    200: (5, 20),  # 5*20*2 = 200
}

print("Grid configurations:")
for n, (nx, ny) in grid_configs.items():
    print(f"  N={n}: {nx}x{ny} grid with 2 base trees = {nx*ny*2} trees")

Grid configurations:
  N=72: 6x6 grid with 2 base trees = 72 trees
  N=100: 5x10 grid with 2 base trees = 100 trees
  N=110: 5x11 grid with 2 base trees = 110 trees
  N=144: 6x12 grid with 2 base trees = 144 trees
  N=156: 6x13 grid with 2 base trees = 156 trees
  N=196: 7x14 grid with 2 base trees = 196 trees
  N=200: 5x20 grid with 2 base trees = 200 trees


In [6]:
# Try lattice optimization for each target N
lattice_results = {}

for n in [72]:  # Start with just N=72 to test
    nx, ny = grid_configs[n]
    print(f"\n=== Optimizing N={n} with {nx}x{ny} grid ===")
    
    best_trees = None
    best_score = float('inf')
    
    # Try multiple random seeds
    for seed in range(5):
        print(f"\nSeed {seed}:")
        trees, score = optimize_base_config(
            n_base=2,
            target_n=n,
            nx=nx,
            ny=ny,
            iterations=500,
            seed=seed
        )
        print(f"  Final score: {score:.6f}")
        
        if score < best_score:
            best_score = score
            best_trees = trees
    
    lattice_results[n] = (best_trees, best_score)
    print(f"\nN={n}: Best lattice score = {best_score:.6f}, Baseline = {baseline_scores[n]:.6f}")
    print(f"  Improvement: {baseline_scores[n] - best_score:.6f}")


=== Optimizing N=72 with 6x6 grid ===

Seed 0:


  Final score: 1.573389

Seed 1:


  Final score: 1.824285

Seed 2:


  Final score: 1.806623

Seed 3:


  Final score: 1.793021

Seed 4:


  Final score: 1.650504

N=72: Best lattice score = 1.573389, Baseline = 0.348559
  Improvement: -1.224830


In [ ]:
# Check if any lattice results are better than baseline
improvements = {}
for n, (trees, score) in lattice_results.items():
    if score < baseline_scores[n]:
        improvements[n] = baseline_scores[n] - score
        print(f"N={n}: IMPROVED by {improvements[n]:.6f}")
    else:
        print(f"N={n}: No improvement (lattice={score:.6f}, baseline={baseline_scores[n]:.6f})")

if not improvements:
    print("\nNo improvements found with lattice approach.")
    print("The baseline is already very well optimized.")

In [ ]:
# Summary
print("\n" + "="*50)
print("LATTICE EXPERIMENT SUMMARY")
print("="*50)
print(f"Tested N values: {list(lattice_results.keys())}")
print(f"Improvements found: {len(improvements)}")
if improvements:
    total_improvement = sum(improvements.values())
    print(f"Total score improvement: {total_improvement:.6f}")
else:
    print("Lattice approach did not improve on baseline.")
    print("The pre-optimized solution is at a very tight local optimum.")